<a href="https://colab.research.google.com/github/younesabdolmalaky/Pistachio/blob/main/InceptionResNetV2_single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
import os
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import pickle
from keras import metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
import random
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:
class TwoChannelDataGenerator(keras.utils.Sequence):

    def __init__(self, image_dir, feature_file, batch_size, img_size):
        self.image_dir = image_dir
        self.feature_file = feature_file
        self.batch_size = batch_size
        self.img_size = img_size

        with open(self.feature_file, 'rb') as f:
            self.features = pickle.load(f)

    def __len__(self):
        return int(np.ceil(len(self.features) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_image_files = self.features[idx * self.batch_size:(idx + 1) * self.batch_size, 48]
        batch_labels = self.features[idx * self.batch_size:(idx + 1) * self.batch_size , 49]

        image_batch = np.zeros((len(batch_image_files), self.img_size, self.img_size, 3), dtype=np.float32)
        for i, image_file in enumerate(batch_image_files):
            image_path = os.path.join(self.image_dir, image_file)
            image = load_img(image_path, target_size=(self.img_size, self.img_size))
            image = img_to_array(image)
            image = image / 255.0
            image_batch[i] =image
        return tf.convert_to_tensor(image_batch, dtype=tf.float32), tf.convert_to_tensor(batch_labels, dtype=tf.float32)
    def on_epoch_end(self):
        indices = np.arange(len(self.features))
        np.random.shuffle(indices)
        self.features = self.features[indices]

image_dir = '/content/drive/MyDrive/Pistachio/train'
feature_file = '/content/drive/MyDrive/Pistachio/vec/train.pickle'
batch_size = 32
image_size = 100

In [ ]:
train_generator = TwoChannelDataGenerator(image_dir , feature_file , batch_size , image_size)

In [ ]:
image_dir = '/content/drive/MyDrive/Pistachio/test'
feature_file = '/content/drive/MyDrive/Pistachio/vec/test.pickle'
batch_size = 32
image_size = 100

In [ ]:
test_generator = TwoChannelDataGenerator(image_dir , feature_file , batch_size , image_size)

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False,
                            input_shape=(100,100,3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(4096,activation="relu")(x)
x = Dense(4096,activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(2096,activation="relu")(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    if layer.trainable==True:
        print(layer)

In [ ]:
callbacks = [
                ModelCheckpoint('model.hdf5',
                                 save_best_only=True)]

In [ ]:
opt = Adam(learning_rate=0.005, clipvalue=1.0)
model.compile(
  loss='binary_crossentropy',
  optimizer=opt,
  metrics=['accuracy', metrics.Precision(), metrics.Recall()]
)

In [ ]:
history = model.fit(train_generator, steps_per_epoch=len(train_generator),epochs=1000,validation_data=test_generator, validation_steps=len(test_generator), callbacks=callbacks)

Epoch 1/1000
54/54 [==============================] - 43s 510ms/step - loss: 317.2905 - accuracy: 0.4881 - precision: 0.3717 - recall: 0.2910 - val_loss: 0.6904 - val_accuracy: 0.5731 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/1000
54/54 [==============================] - 17s 314ms/step - loss: 0.6736 - accuracy: 0.5853 - precision: 0.6667 - recall: 0.0546 - val_loss: 0.9344 - val_accuracy: 0.6636 - val_precision: 0.6466 - val_recall: 0.4674
Epoch 3/1000
54/54 [==============================] - 21s 387ms/step - loss: 0.5261 - accuracy: 0.7507 - precision: 0.7111 - recall: 0.6995 - val_loss: 0.4392 - val_accuracy: 0.8028 - val_precision: 0.7368 - val_recall: 0.8370
Epoch 4/1000
54/54 [==============================] - 22s 415ms/step - loss: 0.4490 - accuracy: 0.7950 - precision: 0.7561 - recall: 0.7664 - val_loss: 0.4179 - val_accuracy: 0.8422 - val_precision: 0.7636 - val_recall: 0.9130
Epoch 5/1000
54/54 [==============================] - 14s 261ms/step - loss: 0.380